The board of the game where C- is a square with crickets (either one or five), E- is an empty space, B- is a bird that eats Lizard

|C1 | E1 | E2|


|E3 | B  | E4|


|E4 | E5 | C5|

Rewards/ penalties for stepping onto squares:
E1-E5 = -1
C1 = +1
C5 = +5 -> WIN THE GAME
B = -10 -> GAME OVER

the q table

rows -> C1 E1 E2 E3 B E4 E5 E6 C5


columns -> LEFT RIGHT UP DOWN

total q states -> 9*4 = 36

In [39]:
import numpy as np
import random

#coefficients used in the Bellman optimality equation
alpha = 0.7
gamma = 0.8

In [86]:
#initalize the q table in case it is not done later
Q_table = np.zeros((9,4))

rewards = np.array([[1, -1, -1], [-1, -10, -1], [-1, -1, 5]])

def reward(pos, a):

    move = pos + a_to_pos(a)
    if borders(move):
        [x , y] = move
        return rewards[x, y]
    else:
        print("Lizard runs into a wall!")

def pos_to_state(pos):
    #converts the coordinates of Lizard into an encoding. E.g. [2,1] is 2*3+1 = 7 which is the 7th square counting from left to right starting from the top (the [0,0] square)
    [x, y] = pos
    state = 3*x + y
    return state

def state_to_pos(state):
    #inversly convert an enocding/state into spatial coordinates
    y = state%3
    x = state//3

    pos = np.array([x, y])

    return pos

def a_to_pos(a):
    #converts a code used for an action into that action. A move is an action of moving Lizard left, right, up or down by one square
    if a == 0:
        return np.array([0, -1])
    elif a == 1:
        return np.array([0, 1])
    elif a == 2:
        return np.array([-1, 0])
    elif a == 3:
        return np.array([1, 0])
    else:
        print("error, wrong action. Out of [0,3] bounds")

def borders(pos):
    #checks if such position is possible in the Lizard game
    [x , y] = pos

    if (2>=x>=0) and (2>=y>=0):
        return True
    else:
        return False

def max_reward(s_new):
    #returns the maximal reward that can be obtained by performing a single action from a given state
    pos = state_to_pos(s_new)
    if borders(pos+ a_to_pos(0)):
        r_left = reward(pos, 0)
    else:
        r_left = 0
    if borders(pos+ a_to_pos(1)):
        r_right = reward(pos, 1)
    else:
        r_right = 0
    if borders(pos+ a_to_pos(2)):
        r_up = reward(pos, 2)
    else:
        r_up = 0
    if borders(pos+ a_to_pos(3)):
        r_down = reward(pos, 3)
    else:
        r_down = 0

    next_reward = max(r_left, r_right, r_up, r_down)

    return next_reward

def update_q_table(pos, a):
    #updates the Q_table
    global Q_table, alpha, gamma
    s = pos_to_state(pos)
    s_next = pos_to_state(pos + a_to_pos(a))
    
    #Bellman optimality equation
    Q_table[s, a] = (1 - alpha) * Q_table[s, a] + alpha * (reward(pos, a) + gamma * max_reward(s_next))



def show_game(pos):
    #a simple way of displaying the current state of the game
    [x, y] = pos
    board = list("⬜⬜⬜\n⬜🔶⬜\n⬜⬜🔷")
    if x == 0:
        board[y] = '⬛'
    if x == 1:
        board[4+y] = '⬛'
    if x == 2:
        board[8+y] = '⬛'
    
    board = "".join(board)

    return board


def train_Lizard(n, start = np.array([2,0]), prompts = False, visualize = False):
    #runs the game n number of times
    global Q_table

    for i in range(n): #specifies how many times the game should be played (until a win or loss)
        over = True
        #at the beginning of the game the lizard always starts in the same position
        pos = start

        if prompts:
            print("starting a new game")

        while over:
            #check if the action is allowed

            a = random.randint(0,3)
            while not borders(pos + a_to_pos(a)):
                a = random.randint(0,3)

            #past this moment an ALLOWED action is generated
            if prompts:
                print("new action generated: "+str(a))
            update_q_table(pos, a)
            #the Q-table gets updated
            if prompts:
                print("the table is succesfully updated")
            pos = pos + a_to_pos(a)
            #the lizard finally moves

            #check if we win or lose
            if (pos == np.array([1,1])).all():
                over = False
                if prompts:
                    print("loss")
            if (pos == np.array([2, 2])).all():
                over = False
                if prompts:
                    print("win")
            
            if visualize:
                print(show_game(pos))
                print("------")

def play_game(pos):
    #let Lizard play the game
    global Q_table

    turn = 0

    while not (pos == np.array([2,2])).all():
        print(show_game(pos))
        print("-------")
        if borders(pos + a_to_pos(np.argmax(Q_table[pos_to_state(pos), :]))):
            #if the action is allowed then Lizard. Action is selected by taking the index of the highest value for a row in the state-action Q_table.
            pos = pos + a_to_pos(np.argmax(Q_table[pos_to_state(pos), :]))
        turn = turn + 1
        if turn == 6:
            #terminate the game after 6 moves no matter what
            break
    
    print(show_game(pos))


In [102]:
#train Lizard
Q_table = np.zeros((9,4))

train_Lizard(10) #Lizard is trained just on 10 games!

In [103]:
print("The Q_table looks like this after training:\n"+str(Q_table))

The Q_table looks like this after training:
[[  0.          -0.19838      0.           0.        ]
 [  0.7          0.           0.         -10.5084    ]
 [  0.           0.           0.           0.        ]
 [  0.         -10.5084       0.973       -0.7       ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           2.99994095  -0.19995626   0.        ]
 [ -0.99757      3.5        -10.5084       0.        ]
 [  0.           0.           0.           0.        ]]


Let's see if Lizard now knows how to play to win!

In [105]:
start = np.array([2,0])
play_game(start)

⬜⬜⬜
⬜🔶⬜
⬛⬜🔷
-------
⬜⬜⬜
⬜🔶⬜
⬜⬛🔷
-------
⬜⬜⬜
⬜🔶⬜
⬜⬜⬛
